In [1]:
"""main.py

Main script for the CMS Run3 analysis
"""
%reset -f
%load_ext autoreload
%autoreload 2
import os
import sys
import pathlib

import numpy as np

import uproot
import awkward as ak
import awkward.numba

import ROOT as rt
from src import CMS_lumi, tdrstyle
from ROOT import TChain, TCanvas, TH1F, TH2F, TF1, TLatex, TGraph, RDataFrame, TLine, TBox
from src.histo_utilities import create_prof1D, create_TGraph, create_TH1D, create_TH2D, std_color_list
from src.helper_functions import (getRecoTime, find_nearest, deltaPhi, deltaR, lor, land, lxor, lnot, asum, canvas,
                                  weight_calc)

from main2 import MuonSystem, get_lat_leg

from main_030223 import H1D, multi_plot, match_clusters, pass_NCSC_NDT, pass_muon_veto, pass_jet_veto, pass_in_time, pass_L1, pass_in_det

########################################################

out_dir = '/home/psimmerl/Documents/CMS/LLP/reports/weekly/mar2/'
data_dir = '/home/psimmerl/Documents/CMS/LLP/data/raw/'

file_db_0p4 = data_dir + 'ggH_HToSSTobbbb_MH-125_MS-15_CTau1000_13p6TeV_1pb_weighted_v4.root'
file_ca_0p4 = data_dir + 'ggH_HToSSTobbbb_MH-125_MS-15_CTau1000_13p6TeV_1pb_weighted_v5.root'
file_ca_0p5 = data_dir + 'ggH_HToSSTobbbb_MH-125_MS-15_CTau1000_13p6TeV_1pb_weighted_v7.root'
file_ca_0p6 = data_dir + 'ggH_HToSSTobbbb_MH-125_MS-15_CTau1000_13p6TeV_1pb_weighted_v6.root'
file_ca_0p8 = data_dir + 'ggH_HToSSTobbbb_MH-125_MS-15_CTau1000_13p6TeV_1pb_weighted_v8.root'
file_ca_1p0 = data_dir + 'ggH_HToSSTobbbb_MH-125_MS-15_CTau1000_13p6TeV_1pb_weighted_v9.root'
# run3_file = data_dir + 'DisplacedJet-EXOCSCCluster_Run2022EFG-PromptReco-v1_goodLumi.root'

files = [file_db_0p4, file_ca_0p4, file_ca_0p5, file_ca_0p6, file_ca_0p8, file_ca_1p0]
labels = [
    'DBSCAN',
    'CA 0.4',
    'CA 0.5',
    'CA 0.6',
    'CA 0.8',
    'CA 1.0',
]
tree_name = 'MuonSystem'
ending = '.png'

dets = ['CSC', 'DT']
pi = rt.TMath.Pi()
met = False
gc = []

is_mc = 'DisplacedJet' not in files[0]

rt.gROOT.SetBatch()

a = tdrstyle.setTDRStyle()
# CMS_lumi.writeExtraText = 0
rt.gStyle.SetOptFit(0)  #1011)

pathlib.Path(out_dir).mkdir(parents=True, exist_ok=True)  # make out directory if it doesn't exist

print(f'Using output directory \'{out_dir}\'')
print(f'Processing tree \'{tree_name}\':')  # from \'{file_in}\':')
print(f'\tTreating file as {"MC" if is_mc else "DATA"}')

fuproots = [uproot.open(ff + ':' + tree_name) for ff in files]
mss_raw = [pass_in_det(fup.arrays()) for fup in fuproots]


Welcome to JupyROOT 6.24/06
Using output directory '/home/psimmerl/Documents/CMS/LLP/reports/weekly/mar2/'
Processing tree 'MuonSystem':
	Treating file as MC


In [ ]:
def draw_csc_z_boxes(hh):
    ymin, ymax = hh.GetMinimum(), hh.GetMaximum()
    xmin, xmax = hh.GetXaxis().GetXmin(), hh.GetXaxis().GetXmax()
    boxes = []

    # xmin, xmax = bins[1], bins[2]
    boxes.append(rt.TBox(xmin,ymin,568,ymax)) #in front of ME11
    boxes.append(rt.TBox(632,ymin,671,ymax)) #between ME11 and ME12
    boxes.append(rt.TBox(724,ymin,789,ymax)) #between ME12 and station2
    boxes.append(rt.TBox(849,ymin,911,ymax)) #between station2 and station3
    boxes.append(rt.TBox(970,ymin,1002,ymax)) #between station3 and station4
    boxes.append(rt.TBox(1073,ymin,xmax,ymax)) #beyond CMS
    for b in boxes:
        b.SetFillColor(15)
        b.SetFillStyle(3001)
        b.Draw('same')


    l = rt.TLatex()
    l.SetTextSize(0.08)
    l.SetTextColor(12)
    l.SetTextAngle(90)
    l.DrawLatex(xmin+80, ymax*0.4, "Steel")

    l2 = rt.TLatex()
    l2.SetTextSize(0.06)
    l2.SetTextColor(13)
    l2.SetTextAngle(90)
    l2.DrawLatex(1110, ymax*0.5, "Beyond CMS")
    text = rt.TLatex()
    text.SetTextSize(0.04)
    text.DrawLatex(570, ymax*1.01, "ME1/1")
    text.DrawLatex(660, ymax*1.01, "ME1/2-3")
    text.DrawLatex(795, ymax*1.01, "ME2")
    text.DrawLatex(920, ymax*1.01, "ME3")
    text.DrawLatex(1015, ymax*1.01, "ME4")

    gc.extend(boxes)# + steel)

In [ ]:
c = canvas(2, 2)

##########################################################################
print('Requiring gLLP to decay in CSC.')
mss = [pass_in_det(ms, 'csc') for ms in mss_raw]
mss = [ms[ms['nGLLP'] == 1] for ms in mss]#den
mss_matched = [pass_NCSC_NDT(match_clusters(ms), ncsc=1, ndt=0) for ms in mss]#num

c.cd(1)
title = ';gLLP Z Decay Vertex [cm];CSC efficiency'
bins = (100, 400, 1100)
nums = [H1D(np.abs(ms['cscRechitCluster_match_gLLP_decay_z']), title, bins=bins) for ms in mss_matched]
dens = [H1D(np.abs(ms['gLLP_decay_vertex_z']), title, bins=bins) for ms in mss]
for num, den in zip(nums, dens):
    num.Divide(den)

_1 = multi_plot(nums, labels, legxy=(0.4,0.25,0.6,0.4))
draw_csc_z_boxes(nums[-1])
gc.extend(nums)


c.cd(2)
title = ';gLLP R Decay Vertex [cm];CSC efficiency'
bins = (100, 0, 800)
nums = [H1D(np.abs(ms['cscRechitCluster_match_gLLP_decay_r']), title, bins=bins) for ms in mss_matched]
dens = [H1D(np.abs(ms['gLLP_decay_vertex_r']), title, bins=bins) for ms in mss]
for num, den in zip(nums, dens):
    num.Divide(den)

_2 = multi_plot(nums, labels, legxy=(0.4,0.25,0.6,0.4))
gc.extend(nums)

##########################################################################
print('Requiring gLLP to decay in DT.')
mss = [pass_in_det(ms, 'dt') for ms in mss_raw]
mss = [ms[ms['nGLLP'] == 1] for ms in mss]#den
mss_matched = [pass_NCSC_NDT(match_clusters(ms), ncsc=0, ndt=1) for ms in mss]#num

c.cd(3)
title = ';gLLP Z Decay Vertex [cm];DT efficiency'
bins = (100, 0, 700)
nums = [H1D(np.abs(ms['dtRechitCluster_match_gLLP_decay_z']), title, bins=bins) for ms in mss_matched]
dens = [H1D(np.abs(ms['gLLP_decay_vertex_z']), title, bins=bins) for ms in mss]
for num, den in zip(nums, dens):
    num.Divide(den)

_3 = multi_plot(nums, labels, legxy=(0.4,0.25,0.6,0.4))
gc.extend(nums)

c.cd(4)
title = ';gLLP R Decay Vertex [cm];DT efficiency'
bins = (100, 0, 800)
nums = [H1D(np.abs(ms['dtRechitCluster_match_gLLP_decay_r']), title, bins=bins) for ms in mss_matched]
dens = [H1D(np.abs(ms['gLLP_decay_vertex_r']), title, bins=bins) for ms in mss]
for num, den in zip(nums, dens):
    num.Divide(den)

_4 = multi_plot(nums, labels
, legxy=(0.4,0.25,0.6,0.4))
gc.extend(nums)
c.Print(out_dir + 'eff_compare_DBSCAN-CA' + ending)



Requiring gLLP to decay in CSC.


: 

: 

In [ ]:
# JET_PT_CUT = 10.0
# MUON_PT_CUT = 20.0
# N_RECHIT_CUT = 90
# jetPt_cut = 50
# tightid = False
# ring_cut = 50

# cut_based = True
# cut_based_version = 'v4'

# intime = True
# DPHI_CUT = 1


# gLLP_csc = {}

# nCscClusters = {}
# selections_cluster = {}
# sel_cluster = {}
# met = {}
# metPhi = {}

# jetMet_dPhiMin = {}
# dphiMet_cluster = {}
# nRechits_sr = {}
# jetMet_dPhiMin30_sr = {}
# cscClusterTimeSpread = {}
# bdt_score = {}
# nCscChambers = {}
# a = {}
# b = {}
# c = {}
# d = {}
# sel_ev = {}
# cluster_index = ''
# nRings = {}
# nLeptons= {}
# cscRechitClusterMuonVetoPt = {}
# cscRechitClusterJetVetoPt = {}

# cscRechitClusterTime = {}
# cscRechitClusterPhi = {}
# cscRechitClusterEta = {}
# cscClusterSize = {}
# cscRechitClusterNStation = {}
# cscRechitClusterMaxStation = {}
# cscRechitClusterDPhiMet = {}
# cscRechitClusterMe11Ratio = {}
# cscRechitClusterMe12Ratio = {}
# cscRechitClusterMe11 = {}
# cscRechitClusterMe12 = {}
# cscRechitClusterDphi = {}

# deltaRCluster = {}
# nDtSectors = {}
# metPhi = {}
# nCscClusters = {}
# nDtClusters = {}
# evtNum = {}
# runNum = {}
# lumiNum = {}
# dtRechitClusterDphi = {}
# nDtWheels25 = {}
# nDTRechitsNoiseSec7 = {}
# nDTRechitsNoiseSec8 = {}
# nDTRechitsNoiseSec9 = {}
# nDTRechitsNoiseSec10 = {}
# nDtStations25 = {}


# dtRechitClusterDPhiMet = {}
# dtRechitClusterNSegmentStation1 = {}
# dtRechitClusterNSegmentStation2 = {}
# dtRechitClusterNSegmentStation3 = {}
# dtRechitClusterNSegmentStation4 = {}
# dtRechitClusterMuonVetoPt = {}
# dtRechitClusterJetVetoPt = {}
# dtRechitClusterPhi = {}
# dtRechitClusterEta = {}
# dtClusterSize = {}
# dtRechitClusterTime = {}
# dtRechitClusterNStation = {}
# dtRechitClusterDPhiMet = {}
# dtRechitClusterMaxStation = {}
# dtRechitClusterMaxStationRatio = {}
# dtRechitClusterNOppositeSegStation1 = {}
# dtRechitClusterNOppositeSegStation2 = {}
# dtRechitClusterNOppositeSegStation3 = {}
# dtRechitClusterNOppositeSegStation4 = {}
# dtRechitClusterMuonVetoLooseId = {}
# nCosmic = {}
# cosmicTwoLegClusterChi2Reduced = {}
# cosmicTwoLegCluster2NStation = {}
# cosmicTwoLegCluster1NStation = {}
# cosmicTwoLegCluster2Index = {}
# cosmicTwoLegCluster1Index = {}
# cscRechitClusterNSegmentStation1 = {}
# cscRechitClusterNSegmentStation2 = {}
# cscRechitClusterNSegmentStation3 = {}
# cscRechitClusterNSegmentStation4 = {}
# cscRechitClusterMuonVetoLooseId = {}
# dtRechitClusterNSegStation1 = {}
# dtRechitClusterNSegStation2 = {}
# dtRechitClusterNSegStation3 = {}
# dtRechitClusterNSegStation4 = {}
# cscRechitClusterMuonVetoGlobal = {}
# cscRechitClusterMuonVetoLooseId = {}
# dtRechitClusterMuonVetoGlobal = {}
# dtRechitClusterMuonVetoLooseId = {}
# dtRechitCluster_match_RPChits = {}
# dtRechitClusterMetEENoise_dPhi = {}
# dtRechitClusterZ = {}
# clusterDphi = {}
# nMe11 = {}
# nJets = {}
# deltaRCluster = {}
# jetMet_dPhiMin = {}
# jetPt = {}
# cond = {}
# category = 2
# #2csc, 2dt, 1csc+1dt
# region = 'signal'
# # region = 'control'
# # region = 'punchthrough'
# for k, T in tree.items():
        
#     ########### SELECTION: CLUSTERS ############
      
   
#     sel_csccluster = np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeWeighted') < 12.5, \
#                                                                          T.array('cscRechitCluster' + cluster_index + 'TimeWeighted') > -5)

#     sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitCluster' + cluster_index + 'TimeSpreadWeightedAll')<20)
#     sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt')<30)
# #     sel_csccluster = np.logical_and(sel_csccluster, np.logical_not(np.logical_and(T.array('cscRechitClusterMuonVetoPt') >= 30, T.array('cscRechitClusterMuonVetoGlobal'))))
#     sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitClusterGlobalMuonVetoPt') < 30)


#     if region == 'control': sel_csccluster = np.logical_and(sel_csccluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi'))>=1.2)
#     elif region == 'signal': sel_csccluster = np.logical_and(sel_csccluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi'))<1.2)
#     if region == 'punchthrough':sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitCluster' + cluster_index + 'Me11Ratio')>=1.0)
#     else: sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitCluster' + cluster_index + 'Me11Ratio')<1.0)

    
#     sel_dtcluster = np.abs(T.array('dtRechitClusterJetVetoPt')) < 50
# #     sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMuonVetoPt') >= 10, T.array('dtRechitClusterMuonVetoLooseId'))))
#     sel_dtcluster = np.logical_and(sel_dtcluster, T.array('dtRechitClusterLooseIdMuonVetoPt') < 10)


#     cut = 5
#     station = (T.array('dtRechitClusterNSegmentStation1')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>cut).astype(int)\
# +(T.array('dtRechitClusterNSegmentStation3')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>cut).astype(int)

#     max_station = np.maximum(np.maximum(np.maximum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
#     min_station = np.minimum(np.minimum(np.minimum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))

#     sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_or(station<4, min_station/max_station<0.4)) #remove if both clusters are 4 stations

#     sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(T.array('dtRechitClusterNoiseVeto'))) #remove if both clusters are 4 stations


# #     noise_2016 = np.logical_and(T.array('dtRechitClusterMaxStation') == 2, T.array('dtRechitClusterWheel') == 1)
# #     noise_2016 = np.logical_and(noise_2016, T.array('dtRechitClusterPhi') > math.pi/12)
# #     noise_2016 = np.logical_and(noise_2016, T.array('dtRechitClusterPhi') < math.pi/4)
# #     sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(noise_2016))
    
#     if region == 'punchthrough':sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_and(T.array('dtRechitClusterMaxStation')==1, T.array('dtRechitClusterMaxStationRatio')>=0.9))
#     else:sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMaxStation')==1, T.array('dtRechitClusterMaxStationRatio')>=0.9)))

#     if region == 'control': sel_dtcluster = np.logical_and(sel_dtcluster, np.abs(T.array('dtRechitClusterMetEENoise_dPhi')) >= 1)
#     elif region == 'signal': sel_dtcluster = np.logical_and(sel_dtcluster, np.abs(T.array('dtRechitClusterMetEENoise_dPhi')) < 1)


#      ###################### cosmic muon veto #############
#     sel_cosmic = np.logical_and(T.array('dtRechitClusterNOppositeSegStation1')>0, T.array('dtRechitClusterNOppositeSegStation2')>0)
#     sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation3')>0)
#     sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation4')>0)
#     sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation1')+T.array('dtRechitClusterNOppositeSegStation2')+\
#                                T.array('dtRechitClusterNOppositeSegStation3')+T.array('dtRechitClusterNOppositeSegStation4')>=6)
#     nstation = (T.array('dtRechitClusterNSegmentStation1')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>1).astype(int)\
#     +(T.array('dtRechitClusterNSegmentStation3')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>1).astype(int)
    
#     sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(nstation>=3, sel_cosmic)))
#     print(np.count_nonzero(sel_dtcluster.sum()))

# ########### SELECTION: JETS ############
    
#     sel_jet = np.logical_and(T.array('jetPt') > 30, np.abs(T.array('jetEta')) < 2.4 )
#     sel_jet = np.logical_and(sel_jet, T.array('jetTightPassId'))

            
# ########### SELECTION: SPIKE IN DT ############
    
#     spike = np.logical_and( T.array('nDTRechitsSector')[:,0,0,7]>50,  T.array('nDTRechitsSector')[:,0,0,7]+T.array('nDTRechitsSector')[:,0,0,8]+T.array('nDTRechitsSector')[:,0,0,9]>120)
#     spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,8]>25)
#     spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,9]>10)

# ########### SELECTION: EVENTS ############
#     hlt = T['HLTDecision'].array()
#     # select only triggered events
#     sel_ev[k] = T.array('METNoMuTrigger')
#     sel_ev[k] = np.logical_and(sel_ev[k], (T.array('nDtRings')+T.array('nCscRings'))<10)
#     sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metEENoise') > 200)
#     sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
#     sel_ev[k] = np.logical_and(sel_ev[k],T.array('Flag2_all'))
#     sel_ev[k] = np.logical_and(sel_ev[k],np.logical_not(spike))

    

#     if category == 0:
#         sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() == 2)
#         sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum() == 0)
#     elif category == 1:
#         sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum() == 2)
#         sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() == 0)
#     else:
#         sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() == 1)
#         sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum() == 1)

    
# ########### BRANCHES ############ 
#     if category == 0:
#         cond[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
#                         T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,1], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,1])<2
#         cscClusterSize[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][cond[k]]

#         cscRechitClusterDphi[k] =  deltaPhi(T.array('cscRechitClusterPhi')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitClusterPhi')[sel_csccluster][sel_ev[k]][:,1])[cond[k]]



#     elif category == 1:

#         dtRechitClusterDphi[k] =  deltaPhi(T.array('dtRechitClusterPhi')[sel_dtcluster][sel_ev[k]][:,0], T.array('dtRechitClusterPhi')[sel_dtcluster][sel_ev[k]][:,1])
#         dtClusterSize[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]]
# #         dtRechitClusterTime[k] =  T.array('dtRechitCluster_match_RPCBx_dPhi0p5')[sel_dtcluster][sel_ev[k]]
# # #         dtRechitClusterNStation[k] =  T.array('dtRechitClusterNStation10')[sel_dtcluster][sel_ev[k]]
# #         dtRechitClusterDPhiMet[k] =  T.array('dtRechitClusterMetEENoise_dPhi')[sel_dtcluster][sel_ev[k]]
# #         dtRechitClusterMaxStation[k] =  T.array('dtRechitClusterMaxStation')[sel_dtcluster][sel_ev[k]]
# #         dtRechitClusterMaxStationRatio[k] =  T.array('dtRechitClusterMaxStationRatio')[sel_dtcluster][sel_ev[k]]
# #         dtRechitClusterMuonVetoLooseId[k] =  T.array('dtRechitClusterMuonVetoLooseId')[sel_dtcluster][sel_ev[k]]
# #         dtRechitClusterNSegStation1[k] = T.array('dtRechitClusterNSegStation1')[sel_dtcluster][sel_ev[k]]
# #         dtRechitClusterNSegStation2[k] = T.array('dtRechitClusterNSegStation2')[sel_dtcluster][sel_ev[k]]
# #         dtRechitClusterNSegStation3[k] = T.array('dtRechitClusterNSegStation3')[sel_dtcluster][sel_ev[k]]
# #         dtRechitClusterNSegStation4[k] = T.array('dtRechitClusterNSegStation4')[sel_dtcluster][sel_ev[k]]

#     else:
#         cond[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
#                         T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster][sel_ev[k]][:,0], T.array('dtRechitCluster' + cluster_index + 'Phi')[sel_dtcluster][sel_ev[k]][:,0])<2.5

# #         dtRechitClusterNSegmentStation1[k]=  T.array('dtRechitClusterNSegmentStation1')[sel_dtcluster][sel_ev[k]][:,0]
# #         dtRechitClusterNSegmentStation2[k]=  T.array('dtRechitClusterNSegmentStation2')[sel_dtcluster][sel_ev[k]][:,0]
# #         dtRechitClusterNSegmentStation3[k]=  T.array('dtRechitClusterNSegmentStation3')[sel_dtcluster][sel_ev[k]][:,0]
# #         dtRechitClusterNSegmentStation4[k]=  T.array('dtRechitClusterNSegmentStation4')[sel_dtcluster][sel_ev[k]][:,0]
# #         dtRechitCluster_match_RPChits[k]=  T.array('dtRechitCluster_match_RPChits_dPhi0p5')[sel_dtcluster][sel_ev[k]][:,0]

# #         dtRechitClusterMetEENoise_dPhi[k]=  T.array('dtRechitClusterMetEENoise_dPhi')[sel_dtcluster][sel_ev[k]][:,0]

#         dtRechitClusterPhi[k] = T.array('dtRechitClusterPhi')[sel_dtcluster][sel_ev[k]][cond[k]][:,0]
# #         dtRechitClusterEta[k] = T.array('dtRechitClusterEta')[sel_dtcluster][sel_ev[k]][:,0]
#         dtClusterSize[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][cond[k]][:,0]
#         cscRechitClusterPhi[k] = T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][cond[k]][:,0]
#         cscClusterSize[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][cond[k]][:,0]

        
# #         deltaRCluster[k] = deltaR(dtRechitClusterEta[k], dtRechitClusterPhi[k],cscRechitClusterEta[k], cscRechitClusterPhi[k])
        
        
# #         cscRechitClusterDPhiMet[k] = np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi'))[sel_csccluster][sel_ev[k]][:,0]
# #         dtRechitClusterDPhiMet[k] = np.abs(T.array('dtRechitClusterMetEENoise_dPhi'))[sel_dtcluster][sel_ev[k]][:,0]
        
#         clusterDphi[k] =  deltaPhi(dtRechitClusterPhi[k], cscRechitClusterPhi[k])



    
#     jetMet_dPhiMin[k] = np.abs(T.array('jetMet_dPhiMin'))[sel_ev[k]]
#     jetPt[k] = T.array('jetPt')[sel_jet][sel_ev[k]]
        
#     nCosmic[k] = sel_cosmic.sum()[sel_ev[k]]
        
#     metPhi[k] = T.array('metPhiEENoise')[sel_ev[k]]

#     nDtWheels25[k] = T.array('nDtWheels25')[sel_ev[k]]

#     nDtStations25[k] = T.array('nDtStations25')[sel_ev[k]]

#     nDTRechitsNoiseSec7[k] = T.array('nDTRechitsSector')[:,0,0,6][sel_ev[k]]
#     nDTRechitsNoiseSec8[k] =  T.array('nDTRechitsSector')[:,0,0,7][sel_ev[k]]
#     nDTRechitsNoiseSec9[k] =  T.array('nDTRechitsSector')[:,0,0,8][sel_ev[k]]
#     nDTRechitsNoiseSec10[k] =  T.array('nDTRechitsSector')[:,0,0,9][sel_ev[k]]

#     nDtSectors[k] = np.sum(np.reshape(T.array('nDTRechitsSector')>=3, (-1,4*5*12)), axis = 1)[sel_ev[k]]
#     evtNum[k] = T.array('evtNum')[sel_ev[k]]
#     runNum[k] = T.array('runNum')[sel_ev[k]]
#     lumiNum[k] = T.array('lumiSec')[sel_ev[k]]
#     nLeptons[k] = T.array('nLeptons')[sel_ev[k]]
#     sel_jet = np.logical_and(T.array('jetPt') > jetPt_cut, np.abs(T.array('jetEta')) < 2.4 )
    
#     sel_jet = np.logical_and(sel_jet, T.array('jetTightPassId'))
#     nJets[k] = sel_jet.sum()[sel_ev[k]]

In [ ]:
# a = {}
# b = {}
# c = {}
# d = {}
# # region = 'control'
# # region = 'signal'

# if region == 'signal':
#     if category == 0:
#         N_RECHIT1_MAX = 100
#         N_RECHIT2_MAX = 100
#     elif category == 1:
#         N_RECHIT1_MAX = 80
#         N_RECHIT2_MAX = 80
#     else:
#         N_RECHIT1_MAX = 80
#         N_RECHIT2_MAX = 100
#     if category == 2: cuts = np.arange(60,100,10)
#     else: cuts = np.arange(60,N_RECHIT1_MAX,5)

# else:
#     if category == 2: cuts = np.arange(60,110,10)
#     else: cuts = np.arange(60,160,5)
#     N_RECHIT1_MAX = 10000000000
#     N_RECHIT2_MAX = 10000000000


# n_ev = 5000

# print(cuts)
# print(region, category)
# for k in tree.keys():

#     if category == 0:
#         cond= np.abs(cscRechitClusterDphi[k])<2.4
#         var1 = cscClusterSize[k][cond][:,0]
#         var2 = cscClusterSize[k][cond][:,1]
#     elif category == 1:
#         cond =  np.abs(dtRechitClusterDphi[k])<2
#         var1 = dtClusterSize[k][cond][:,0]
#         var2 = dtClusterSize[k][cond][:,1]
#     else: # CSC-DT
#         cond = np.abs(clusterDphi[k])<2.2
#         var1 = dtClusterSize[k][cond]
#         var2 = cscClusterSize[k][cond]
#     for N_RECHIT_CUT1 in cuts:
#         for N_RECHIT_CUT2 in cuts:
#             if category <2 and not N_RECHIT_CUT2 == N_RECHIT_CUT1:continue
#             if N_RECHIT_CUT1>= N_RECHIT1_MAX:continue
#             if N_RECHIT_CUT2>= N_RECHIT2_MAX:continue


            
#             a[k] = np.count_nonzero(np.logical_and(np.logical_and(var1<N_RECHIT1_MAX, var1>=N_RECHIT_CUT1), np.logical_and(var2<N_RECHIT2_MAX, var2>=N_RECHIT_CUT2)))
#             b[k] = np.count_nonzero(np.logical_and(var1<N_RECHIT_CUT1, np.logical_and(var2<N_RECHIT2_MAX, var2>=N_RECHIT_CUT2)))
#             c[k] = np.count_nonzero(np.logical_and(var1<N_RECHIT_CUT1, var2<N_RECHIT_CUT2)) #both less
#             d[k] =  np.count_nonzero(np.logical_and(np.logical_and(var1<N_RECHIT1_MAX, var1>=N_RECHIT_CUT1), var2<N_RECHIT_CUT2))
            
            
#             if category == 2:
#                 if d[k]==0 or b[k] == 0 or c[k] == 0:
#                     if c[k] == 0:pred = 10000
#                     else: pred = b[k]*d[k]/c[k]
#                     print(N_RECHIT_CUT1,'\t', N_RECHIT_CUT2, '\t',a[k],'\t',b[k],'\t',c[k],'\t',d[k],'\t', round(pred, 2), '\t',\
#                       0.0, '\t', 0.0)

#                 else:
#                     pred = b[k]*d[k]/c[k]
#                     unc_pred = (1./b[k] + 1./d[k] + 1./c[k])**0.5*pred

#                     if math.isnan(unc_pred): z_value = float("nan")
#                     else:
#                         mu = np.random.normal(pred, unc_pred, n_ev)
#                         p_value = 0.0
#                         for i in mu:
#                             if i < 0:continue
#                             n = np.random.poisson(i, n_ev)
#                             p_value += np.count_nonzero(n>=a[k])
#                         p_value = p_value/n_ev**2
#                         z_value = abs(norm.ppf(p_value))
#                     print(N_RECHIT_CUT1,'\t', N_RECHIT_CUT2, '\t',a[k],'\t',b[k],'\t',c[k],'\t',d[k],'\t', round(pred, 2), '\t',\
#                           round(unc_pred, 2), '\t', round(z_value,2))
# #                     print('{} & {} & {} & {} & {} & {} & {} $\pm$ {} \\\ '.format(N_RECHIT_CUT1, N_RECHIT_CUT2,a[k], b[k], c[k], d[k], round(pred,2), round(unc_pred,2)))
#             else:
#                 if d[k]+b[k] == 0 or c[k] == 0:
#                     if c[k] == 0:pred = 10000
#                     else: pred = ((b[k]+d[k])/2/c[k])**2*c[k]
#                     print(N_RECHIT_CUT1,'\t', N_RECHIT_CUT2, '\t',a[k],'\t',b[k]+d[k],'\t',c[k], '\t', round(pred, 2), '\t',\
#                       0.0, '\t', 0.0)
#                 else:
#                     pred = ((b[k]+d[k])/2/c[k])**2*c[k]
#                     unc_pred = (4/(b[k]+d[k]) + 1./c[k])**0.5*pred

#                     if math.isnan(unc_pred): z_value = float("nan")
#                     else:
#                         mu = np.random.normal(pred, unc_pred, n_ev)
#                         p_value = 0.0
#                         for i in mu:
#                             if i < 0:continue
#                             n = np.random.poisson(i, n_ev)
#                             p_value += np.count_nonzero(n>=a[k])
#                         p_value = p_value/n_ev**2
#                         z_value = abs(norm.ppf(p_value))
#                     print(N_RECHIT_CUT1,'\t', N_RECHIT_CUT2, '\t',a[k],'\t',b[k]+d[k],'\t',c[k],'\t',round(pred, 2), '\t',\
#                           round(unc_pred, 2), '\t', round(z_value,2))

In [ ]:
# import importlib
# importlib.reload(sys.modules['histo_utilities'])
# from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot


# k = 'data'


        
# if category == 0:
#     var = [cscClusterSize[k][:,0], cscClusterSize[k][:,1]]
#     varName = ['cscRechitClusterSize1', 'cscRechitClusterSize2']
#     xaxis = ['N_{csc}','N_{csc}']
# elif category == 1:
#     var = [dtClusterSize[k][:,0], dtClusterSize[k][:,1]]
#     varName = ['dtRechitClusterSize1', 'dtRechitClusterSize2']
#     xaxis = ['N_{dt}','N_{dt}']
# else:
#     var = [cscClusterSize[k], dtClusterSize[k]]
#     varName = ['cscRechitClusterSize1', 'dtRechitClusterSize1']
#     xaxis = ['N_{csc}','N_{dt}']
# cut_index = 0
# leg = rt.TLegend(0.6,0.77,0.90,0.92)

# leg.SetTextSize(0.03)
# leg.SetBorderSize(0)
# leg.SetEntrySeparation(0.01)
# c = rt.TCanvas('c','c', 800, 800)
# rt.gStyle.SetOptFit(1011)
# h = {}
# r = {}
# for j,v in enumerate(var):

#         if j == 1:continue
#         print("category:", category)
#         print("region: ", region)

#         cond = np.logical_and(var[0]<100, var[1]<100)
        
# #         else: cond = np.logical_and(var[0]<80, var[1]<80)
# #         cond = np.abs(jetMet_dPhiMin[k])>0.6
#         if category <2: h[varName[j]] = create_TH1D( list(var[0][cond])+list(var[1][cond]), axis_title=[xaxis[j], 'Events'], name=k, binning=[5,50,100])
#         else:h[varName[j]] = create_TH1D( v[cond], axis_title=[xaxis[j], 'Events'], name=k, binning=[5,50,100])
#         leg.AddEntry(h[varName[j]], varName[j])
#         h[varName[j]].GetXaxis().SetLabelSize(0.04)
#         h[varName[j]].SetLineColor(std_color_list[0])
#         r[varName[j]] = h[varName[j]].Fit('expo', 'LRSQ+', '', 50,100)
#         a = r[varName[j]].Parameter(1)
#         print(a)
#         print('NRECHIT_CUT, Integral, efficiency, efficiencyUp, efficiencyDown, Integral*Eff^2')
#         for N_RECHIT_CUT in [70, 80,90,100]:
#             a = r[varName[j]].Parameter(1)
#             eff = math.exp(N_RECHIT_CUT*a)/(math.exp(50*a)-math.exp(100*a))
#             a = r[varName[j]].Parameter(1) + r[varName[j]].Error(1)
#             effUp = math.exp(N_RECHIT_CUT*a)/(math.exp(50*a)-math.exp(100*a))
#             a = r[varName[j]].Parameter(1) - r[varName[j]].Error(1)
#             effDown = math.exp(N_RECHIT_CUT*a)/(math.exp(50*a)-math.exp(100*a))
#             if category <2: print(N_RECHIT_CUT, h[varName[j]].Integral()/2, eff, effUp, effDown, h[varName[j]].Integral()/2*eff**2 )
#             else: print(N_RECHIT_CUT, h[varName[j]].Integral(), eff, effUp, effDown, h[varName[j]].Integral()*eff**2 )
#         for N_RECHIT_CUT in [70, 80,90,100]:
#             a = r[varName[j]].Parameter(1)
#             eff = (math.exp(50*a)-math.exp(N_RECHIT_CUT*a))/(math.exp(50*a)-math.exp(100*a))
#             a = r[varName[j]].Parameter(1) + r[varName[j]].Error(1)
#             effUp = (math.exp(50*a)-math.exp(N_RECHIT_CUT*a))/(math.exp(50*a)-math.exp(100*a))
#             a = r[varName[j]].Parameter(1) - r[varName[j]].Error(1)
#             effDown = (math.exp(50*a)-math.exp(N_RECHIT_CUT*a))/(math.exp(50*a)-math.exp(100*a))
#             if category <2: print(N_RECHIT_CUT, h[varName[j]].Integral()/2, eff, effUp, effDown, h[varName[j]].Integral()/2*eff**2 )
#             else: print(N_RECHIT_CUT, h[varName[j]].Integral(), eff, effUp, effDown, h[varName[j]].Integral()*eff**2 )
#         h[varName[j]].GetFunction("expo").SetLineWidth(2)
#         h[varName[j]].Draw('same E1')


# c.SetRightMargin(0)
# c.SetLogy()
# # leg.Draw()
# #     c.SaveAs('/storage/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/plots/MuonSystem_Analysis/abcdVar/v1p15_'+k+'_'+str(data_year)+'_'+varName[j]+'.png')
# c.Draw()
# # print(time.time()-start_t)
# # expo: Exponential function with two parameters: f(x) = exp(p0+p1*x)